<h1 style="text-align: center;margin:-7px">Intro to Natural Language Processing</h1>
<h2 style="text-align: center;">Assignment 1</h2>
<h3 style="text-align: center;margin:7px">Student ID: 20231272</h3>
<h3 style="text-align: center;margin:3px">Student Name: Kollapudi Nagendra Dheeraj</h3>

# Assignment 1

This assignment will involve the creation of a spellchecking system and an evaluation of its performance. You may use the code snippets provided in Python for completing this or you may use the programming language or environment of your choice

Please start by downloading the corpus `holbrook.txt` from Blackboard

The file consists of lines of text, with one sentence per line. Errors in the line are marked with a `|` as follows

    My siter|sister go|goes to Tonbury .
    
In this case the word 'siter' was corrected to 'sister' and the word 'go' was corrected to 'goes'.

In some places in the corpus two words maybe corrected to a single word or one word to a multiple words. This is denoted in the data using underscores e.g.,

    My Mum goes out some_times|sometimes .
    
For the purpose of this assignment you do not need to separate these words, but instead you may treat them like a single token.

*Note: you may use any functions from NLTK to complete the assignment. It should not be necessary to use other libraries and so please consult with us if your solution involves any other external library. If you use any function from NLTK in Task 6 please include a brief description of this function and how it contributes to your solution.*

## Task 1 (10 Marks)

Write a parser that can read all the lines of the file `holbrook.txt` and print out for each line the original (misspelled) text, the corrected text and the indexes of any changes. The indexes refers to the index of the words in the sentence. In the example given, there is only an error in the 10th word and so the list of indexes is [9]. It is not necessary to analyze where the error occurs inside the word.

Then split your data into a test set of 100 lines and a training set.

In [1]:
#!pip install nltk
import nltk

from nltk.tokenize import word_tokenize
from nltk.corpus import words
from nltk.corpus import wordnet as wn
from nltk.tag import pos_tag
from nltk.collocations import *

import string

In [2]:
# Write your code here
def parser(txt_file_name):
    
    #Reading the txt file and extracting all the lines
    lines = open(txt_file_name).readlines()
    data = []
    
    #Based on the assert function below, we have to break the list of sentences into list of words via tokenizing the sentences
    worded_list = [word_tokenize(sent) for sent in lines]
    
    #Now we have to search for | and create assert style, dictionary structure with a left | value in original and right | value
    #in corrected along with keeping track of the index where | is present.
    for word_list in worded_list:
        #First search inside the list for |
        ##As | is not a individual value like ['friend', 'is', 'sic', '|', 'sick'] inside a list 
        ###but is embedded in a element of list ['friend', 'is', 'sic|sick'] so direct in wont work, but double for is required.
        
        #Creating these varibables for each list element present inside main list, as below assert requirement is in such fashion.
        original_string = []
        corrected_string = []
        correction_index = []
        count_index = 0
        for word in word_list:
            if '|' in word:
                
                #Then split the word for |
                splitted_context = word.split('|')
                
                # Left aka wrong word is stored to original.
                original_string.append(splitted_context[0])
                
                # Right aka correct word is stored to corrected.
                corrected_string.append(splitted_context[1])
                
                #Noting down the correction index.
                correction_index.append(count_index)
        
            else:
                # If there is no '|' in word present inside the list, then store that word original and corrected listed strings.
                original_string.append(word)
                corrected_string.append(word)

            #Keep track of index until if correction index is found
            count_index += 1

        #Loading to original and corrected listed strings along with the correction_index to dictionary.      
        dictionary = {'original': original_string, 'corrected': corrected_string, 'indexes': correction_index}
        data.append(dictionary)
    return data
    
data = parser("holbrook.txt") 

assert(data[2] == {
   'original': ['I', 'have', 'four', 'in', 'my', 'Family', 'Dad', 'Mum', 'and', 'siter', '.'], 
   'corrected': ['I', 'have', 'four', 'in', 'my', 'Family', 'Dad', 'Mum', 'and', 'sister', '.'], 
   'indexes': [9]
})

The counts and assertions given in the following sections are based on splitting the training and test set as follows

In [3]:
test = data[:100]
train = data[100:]

## **Task 2** (10 Marks): 
Calculate the frequency (number of occurrences), *ignoring case*, of all words and their unigram probability from the corrected *training* sentences.

*Hint: use `Counter` to implement this so it may be called many times*

In [4]:
#ReStructuring the Dictionary of elements into list of words so that it can be used for processing
train_corrected = [dictionary_element['corrected'] for dictionary_element in train]

#Combing the list into one huge string so that the word_tokenize function could process it.
train_corrected = [" ".join(dictionary_element) for dictionary_element in train_corrected]
train_set_fit_for_processing = " ".join(train_corrected)

In [5]:
#Tokenize the sentences into words
tokenized_train_set_fit_for_processing = word_tokenize(train_set_fit_for_processing)
length_of_training_set = len(tokenized_train_set_fit_for_processing)

from collections import Counter

#Applied Counter to get values of occurrences of each element in the list
fdist = Counter(tokenized_train_set_fit_for_processing)

#Extracting unique words using keys() so that it can be used inside preprocess_check and get_candidates
unique_words = list(fdist.keys())

def unigram(word):
    return fdist[word]
    

def prob(word):
    return unigram(word)/length_of_training_set

# Test your code with the following
assert(unigram("me")==87)
assert(round(prob("me"),3)==0.004)

## **Task 3** (15 Marks): 
[Edit distance](https://en.wikipedia.org/wiki/Edit_distance) is a method that calculates how similar two strings are to one another by counting the minimum number of operations required to transform one string into the other. There is a built-in implementation in NLTK that works as follows:


In [6]:
from nltk.metrics.distance import edit_distance

# Edit distance returns the number of changes to transform one word to another
print(edit_distance("hello", "hi"))

4


Write a function that calculates all words with *minimal* edit distance to the misspelled word. You should do this as follows

1. Collect the set of all unique tokens in `train`
2. Find the minimal edit distance, that is the lowest value for the function `edit_distance` between `token` and a word in `train`
3. Output all unique words in `train` that have this same (minimal) `edit_distance` value

*Do not implement edit distance, use the built-in NLTK function `edit_distance`*

In [7]:
def get_candidates(token):
    #Creating a dictionary of words as key and their edit_distance as value for each unique_word
    word_distances = dict((word, edit_distance(word, token)) for word in unique_words)
    
    #Sort the dictionary based on values from min to max and get the first element
    minimal_dist = sorted(word_distances.values())[0]
    
    #Filter out and get those words that have a minimal distance
    return list(filter(lambda k: word_distances[k] == minimal_dist, word_distances.keys()))
        
# Test your code as follows
assert get_candidates("minde") == ['mine', 'mind']

## Task 4 (15 Marks):

Write a function that takes a (misspelled) sentence and returns the corrected version of that sentence. The system should scan the sentence for words that are not in the dictionary (set of unique words in the training set) and for each word that is not in the dictionary choose a word in the dictionary that has minimal edit distance and has the highest *unigram probability*. 

*Your solution to this should involve `get_candidates`*


In [8]:
def correct(sentence):
    corrected = []
   
    for i in sentence:
        # If word not in unique word the calculate suggested words with minimal distance
        if i.lower() not in unique_words:
            
            #Fetch all the candidates that match the candidate
            candidates = get_candidates(i)
            
            #If there are more than 1 values that have same similarity
            if len(candidates) > 1:
                probabilities = {}
                
                #Retrieve their probabilites as per the trained model
                for candidate in candidates:
                    probabilities[candidate] = prob(candidate)
                
                #Sort Dictionary values but in reverse as we want the highest and return top element
                highest_prob = sorted(probabilities.values(), reverse=True)[0]
                
                #Filter Dictionary whose values match the highest prob one and return only the keys
                candidates_with_highest_prob = list(
                    filter(lambda k: probabilities[k] == highest_prob, probabilities.keys()))
                
                #Return the candidate with the highest probabilities
                corrected.append(candidates_with_highest_prob[0])
            else:
                corrected.append(get_candidates(i)[0])

        else:
            corrected.append(i)
    return corrected

assert(correct(["this","whitr","cat"]) == ['this','white','cat'])   

## **Task 5** (10 Marks): 
Using the test corpus evaluate the *accuracy* of your method, i.e., how many words from your system's output match the corrected sentence (you should count words that are already spelled correctly and not changed by the system).

In [13]:
def accuracy(test):
    accuracy = 0
    
    #Iterate through each of the statement
    for _, individual_test_dictionary in enumerate(test):
        corrected_string = individual_test_dictionary['corrected']
        predicted_string = correct(individual_test_dictionary['original'])
        
        #Calculate the accuracy by getting the length of words from system's output and corrected sentence
        #by taking set intersection divided by the length of the corrected string
        accuracy += len([i for i, j in zip(corrected_string, predicted_string) if i == j])/len(corrected_string)
    
    return accuracy

print("Accuracy of the system: ", accuracy(test))

Accuracy of the system:  81.9971390685487


## **Task 6 (35 Marks):**

Consider a modification to your algorithm that would improve the accuracy of the algorithm developed in Task 3 and 4

* You may resources beyond those provided here.
* You must **not use the test data** in this task.
* Provide a short text describing what you intend to do and why. 
* Full marks for this section may be obtained without an implementation, but an implementation is preferred.
* Your implementation should not consist of more than 50 lines of code

Please note this task is marked according to: demonstration of knowledge from the lectures (10), originality and appropriateness of solution (10), completeness of description (10) and technical correctness (5)


### Enhancements done to the algorithm:

* Inorder to enhance the algorithm we need to enhance the accuracy. Based on checking the predicted and corrected for each sentence by looking over training data. I was able to derive 2 ways that help compensate this drop in accuracy. 
    
    
    One way is by reducing accuracy drop which is because of:     
        * Unwanted processing.
            * Correct words, punctation or digits were getting processed
            * Words that are Proper Nouns are getting processed.
         * Solution - Some kind of pre-process checking so that these unnecessary words won't get processed
         
         
    Another way is by increasing accuracy by: 
        * Finding appropriate word based on surrounding words.
            * Finding the likelihood of the current word based on the previous word.
         * Solution - Using nltk package to detect the exact word and send it as a additional list for the system to detect the relevancy via the probabilities from get_candidates() function
       
   * Processing these pain points, few using python packages and rest using NLTK helped rectify the issues with previous correct function that helping in increasing accuracy by a **good** margin.
   
## Unwanted processing:
### Implemented under -> preprocess_check(word):
**Why?:** On viewing few incorrect predictions like [a, at] [guts, nuts] [their, there] etc. It is pretty much clear that some level of unwanted processing is happening as the words on the left are correct and there is no spelling correction needed. It is processing them to values on the right. So in order to stop this a level of pre-process check was implemented to make sure a, guts, their don't enter into get_candidates() function.

#### 1. Unique Words:
 As the algorithm is structured in such a way that it processes any words it sees, so some of the words need not to get processed. So using the unique words trained from the FreqDist from unigram processing, I extracted the keys and added it into preprocess criterion so that these words that match with list do not get processed. Along with the unique_word inclusion of WordNet and isDigit were also added to reduce the processing time by a significant amount.

#### 2. WordNet:
 As Unique words are trained over corpus it might not be sufficient for all the cases, so we need a library in determining whether a word is proper or not. So I used wordnet.words() from WordNet Library, which has a range of values that can help in identifing whether if a word in the sentence is correct, and if so don't consider it for processing.

#### 3. IsDigit:
 Along with the processing of some unnecessary words, I also considered not to process number and special characters as there processing is not needed in the first place, and as they are not part of any wordnet dictionaries. So checking for the presence of numbers, if present then not include it in the processing.
 
#### 4. string.punctation:
 As mentioned above with the interest of time, unnecessary processing these is of no use, so neglecting them as part of special condition as punctation or special characters are not part of wordnet dictionaries. So checking for the presence of special characters, if present then not include it in the processing.
 
#### 5. Proper Nouns:
 Some proper names in the corpus such as "End", "Honda", "JOAN" ,"STALL" is getting processed even though they are right or wrong it shouldn't. So using the nltk.tag.pos_tag to check if the word's part of speech is not starting with 'NN'. This addition proved helpful as the algorithm was able to receive better boost in accuracy. So Implemented the same.

## Finding appropriate word based on surrounding words:
### Implemented under -> likelihood_words(word)
#### 6. BigramCollocation:
 BigramCollocation was used here to find adquate word for the mispelled word considering the likelihood of one word on the left or right from the mispelled word based on the train corpus. So basically I'm sending this extra result to the get_candidates function so that it can help predict a better fit for the mispelled word, but it didn't give any significant boost as the test cases might have had most of the scenarios not seen during BigramCollocationFinder phase. An assert statement was added for better info.

## Some more ways the accuracy could be increased: (Not Implemented)
#### 7. Presence of more Training Data:
 Presence of more data to train for would for sure enhance the accuracy levels. Atleast some overlap between the train and test scenarios would help the BigramCollocation could work better.

#### 8. Presence of some vectorized word denotation for context instead of simple word list:
 As mentioned above, nltk's words.words() contains few confusing words which result incorrect word estimation. Hence, instead of words.words() if presence of some vectorized format to fetch the words based on neighbouring words and considering cosine similarity might result in slight increase in accuracy.
 
#### 10. Consideration of Bigram and Trigram Probabilities:
 Bigram, Trigram and n-gram probabilities could also be introduced to check the most probabilistic word to the mispelled considering the left n words or right n words and passing them to their probabilistic function to check the word with highest index and retrieving it rather than simple unigram.

In [10]:
def preprocess_check(word):
    conditions = []
    
    #Checking if the word is not present in the train_corrected unique set list
    conditions.append(word.lower() not in unique_words)
    
    #Check if word in itself is valid or not
    conditions.append(word not in wn.words())

    #Check if word is not a digit
    conditions.append(not word.isdigit())
    
    #Check if word is a special character
    conditions.append(word not in string.punctuation)
    
    #Check if word is not a noun or proper noun using pos_tag.
    pos_status = pos_tag([word])[0][1]
    conditions.append(not pos_status.startswith('NN'))
    
    return all(conditions)

assert(preprocess_check('JOAN') == False)

Bigram_measures = nltk.collocations.BigramAssocMeasures()
finder = BigramCollocationFinder.from_words(word_tokenize(train_set_fit_for_processing))

def likelihood_words(word):
    return [j for (i, j), v in finder.score_ngrams(Bigram_measures.raw_freq) if i == word]

assert(likelihood_words('Japanese') == ['Honda'])

In [11]:
def enhanced_correct(sentence):
    corrected = []
    
    #Iterate through each word present in the statement
    for i, word in enumerate(sentence):
        
        #Check if word doesn't satisfies the above conditions, if so go ahead with pre-processing.
        ##otherwise directly append to final output.
        if preprocess_check(word):

            #Get all the related candidates from unigram and bigramcollation processing, if the word is wrong
            candidates = get_candidates(word) + likelihood_words(sentence[i-1])
            
            #If there are more candidates
            if len(candidates) > 1:
                probabilities = {}
                
                #Retrieve their probabilites as per the trained model
                for candidate in candidates:
                    probabilities[candidate] = prob(candidate)
                    
                #Sort Dictionary values but in reverse as we want the highest and return top element
                highest_prob = sorted(probabilities.values(), reverse=True)[0]
                
                #Filter Dictionary whose values match the highest prob one and return only the keys
                candidates_with_highest_prob = list(
                    filter(lambda k: probabilities[k] == highest_prob, probabilities.keys()))
                
                #Return the candidate with the highest probabilities
                corrected.append(candidates_with_highest_prob[0])
            
            #Else give the first element
            else:
                corrected.append(candidates[0])

        else:
            corrected.append(word)

    return corrected

## **Task 7 (5 Marks):**

Repeat the evaluation (as in Task 5) of your new algorithm and show that it outperforms the algorithm from Task 3 and 4

In [12]:
def accuracy_mod(test):
    accuracy = 0
    
    #Iterate through each of the statement
    for _, individual_test_dictionary in enumerate(test):
        corrected_string = individual_test_dictionary['corrected']
        predicted_string = enhanced_correct(individual_test_dictionary['original'])
        
        #Calculate the accuracy by getting the length of words from system's output and corrected sentence
        #by taking set intersection divided by the length of the corrected string
        accuracy += len([i for i, j in zip(corrected_string, predicted_string) if i == j])/len(corrected_string)
    
    return accuracy

print("Accuracy of the system: ", accuracy_mod(test), "%")

Accuracy of the system:  88.70299035135751 %


### Enhanced Model Evaluation:

By Implementing all the above mentioned methods of reducing processing and finding alternatives, an accuracy boost from 82% to 88% was seen and resulting in a 6% increase from Task 5 to Task 7.

### Corresponding Results
Accuracy of the modified system:  88.7 %